In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
train_add = pd.read_csv('train_add.csv')
test_add = pd.read_csv('test_add.csv')

In [ ]:
def transform_dataframe(df, date_column):
    # Copy the original dataframe to avoid modifying the original
    transformed_df = df.copy()

    # Convert the date column to datetime type
    transformed_df[date_column] = pd.to_datetime(transformed_df[date_column])

    # Extract day, month, and year from the date column
    transformed_df['day'] = transformed_df[date_column].dt.day
    transformed_df['month'] = transformed_df[date_column].dt.month
    transformed_df['year'] = transformed_df[date_column].dt.year

    # Add day of the week column
    # transformed_df['day_of_week'] = transformed_df[date_column].dt.weekday

    # Remove the original date column
    transformed_df.drop(date_column, axis=1, inplace=True)

    return transformed_df



In [ ]:
train_data = transform_dataframe(train_add, 'period_dt')
test_data = transform_dataframe(test_add, 'period_dt')

In [ ]:
import pandas as pd

def transform_dt_df(df):
    # Copy the original dataframe to avoid modifying the original
    transformed_df = df.copy()

    # Specify the columns to transform
    columns_to_transform = [
                            'VALID_FROM_DTTM_y']

    # Transform the specified columns
    for column in columns_to_transform:
        transformed_df[column] = pd.to_datetime(transformed_df[column], format="%d%b%Y:%H:%M:%S")

        # Split date into day, month, and year
        transformed_df[column + '_day'] = transformed_df[column].dt.day
        transformed_df[column + '_month'] = transformed_df[column].dt.month
        transformed_df[column + '_year'] = transformed_df[column].dt.year

        # Split time into separate columns
        transformed_df[column + '_hour'] = transformed_df[column].dt.hour
        transformed_df[column + '_minute'] = transformed_df[column].dt.minute
        transformed_df[column + '_second'] = transformed_df[column].dt.second

        # Add weekday column
        transformed_df[column + '_weekday'] = transformed_df[column].dt.weekday

        # Ensure dates are no later than the year 2260
        transformed_df.loc[transformed_df[column + '_year'] > 2260, [column + '_day', column + '_month', column + '_year']] = [1, 1, 2260]

    # Remove the original string columns
    transformed_df.drop(columns_to_transform, axis=1, inplace=True)

    return transformed_df



In [ ]:
train_data = transform_dt_df(train_data)
test_data = transform_dt_df(test_data)

In [ ]:
def filter_df(df, D):
    # Convert day, month, and year columns to datetime format
    df['date'] = pd.to_datetime(df[['day', 'month', 'year']])
    df.sort_values(by='date', inplace=True)
    # Group the DataFrame by product_id and find the first occurrence
    first_occurrence = df.groupby(by=['location_id', 'product_id'])['date'].first()
    first_occurrence = pd.DataFrame(first_occurrence)
    first_occurrence.reset_index(inplace=True)
    first_occurrence.rename(columns={'date':'first_date'}, inplace=True)
    # Merge the first occurrence back to the original DataFrame
    df = df.merge(first_occurrence, on=['location_id','product_id'], suffixes=('', '_first'))

    # Calculate the difference in days between subsequent occurrences and the first occurrence
    df['date_diff'] = (df['date'] - df['first_date']).dt.days

    # Filter out the products where the date difference is greater than D
    filtered_df = df[df['date_diff'] <= D].copy()

    # Drop unnecessary columns
    filtered_df.drop(['first_date', 'date_diff'], axis=1, inplace=True)

    return filtered_df

train_data = filter_df(train_data, 7)

In [ ]:
train_data.dropna(inplace=True)

In [ ]:
# добавим процент от начальной цены
train_data.loc[:, 'disc_percent'] = train_data['PRICE_AFTER_DISC'] / train_data['PRICE_REGULAR']
test_data.loc[:, 'disc_percent'] = test_data['PRICE_AFTER_DISC'] / test_data['PRICE_REGULAR']

In [ ]:
# Средние значения ответа
group_all = train_data[['month', 'location_id', 'demand']].groupby(by=['month', 'location_id'], as_index=False).mean()

group_date = train_data[['month', 'demand']].groupby(by=['month'], as_index=False).mean()
train_data = train_data.merge(group_all, on=['month', 'location_id'], how='left', suffixes=('', '_all'))
test_data = test_data.merge(group_all, on=['month', 'location_id'], how='left', suffixes=('', '_all'))
train_data = train_data.merge(group_date, on=['month'], how='left', suffixes=('', '_new'))
test_data = test_data.merge(group_date, on=['month'], how='left', suffixes=('', '_new'))


In [ ]:
train_data = train_data.sort_values(by=['year', 'month', 'day'])
tmp = train_data.groupby(by=['location_id', 'product_id'], as_index=False).cumcount() + 1
train_data.loc[:, 'number_of_week'] = tmp
test_data = test_data.sort_values(by=['year', 'month', 'day'])
tmp = test_data.groupby(by=['location_id', 'product_id'], as_index=False).cumcount() + 1
test_data.loc[:, 'number_of_week'] = tmp


In [ ]:
train_data.drop(columns=['date'], inplace=True)
train_data.drop(columns=['product_id'], inplace=True)
test_data.drop(columns=['product_id'], inplace=True)

In [ ]:
text_features = train_data.columns[(train_data.dtypes == 'object')].tolist()

In [ ]:
from copy import deepcopy
def demand_bucketing(df_column, closest_points):
    column = list(df_column)
    ans = []
    for elem in column:
        if elem > 3:
            ans.append(3)
            continue
        min_val = 100500
        cor_point = 0
        for it2, point in enumerate(closest_points):
            if abs(point - elem) < min_val:
                min_val = abs(point - elem)
                cor_point = point
        ans.append(cor_point)
    return ans

In [ ]:
train_data.loc[:, 'demand_class'] = demand_bucketing(train_data['demand'], [0, 1, 3])

In [ ]:
from catboost import CatBoostClassifier, Pool
X, y = train_data.drop(columns=['demand', 'demand_class']), train_data['demand_class']
data = Pool(X, y, cat_features=text_features)

model_class1 = CatBoostClassifier(silent=True, random_seed=42, eval_metric='MCC', n_estimators=700, depth=6, learning_rate=0.15, l2_leaf_reg=0.5, cat_features=text_features)
model_class1.fit(data)

In [ ]:
from catboost import CatBoostClassifier, Pool
X, y = train_data.drop(columns=['demand', 'demand_class']), train_data['demand_class']
data = Pool(X, y, cat_features=text_features)

model_class2 = CatBoostClassifier(silent=True, random_seed=42, eval_metric='MCC', n_estimators=700, depth=8, learning_rate=0.15, l2_leaf_reg=0.5, cat_features=text_features)
model_class2.fit(data)

In [ ]:
y_pred1 = model_class1.predict(test_data.drop(columns=['demand']))
y_pred2 = model_class2.predict(test_data.drop(columns=['demand']))
y_pred1 = np.where(y_pred1 <= 1, 1, 1.2)
y_pred2 = np.where(y_pred2 <= 1, 1, 1.2)
test_data.loc[:, 'demand'] = (y_pred1 + y_pred2) / 2

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 14))
sns.histplot(test_data['demand'])

In [ ]:
test_data[['id', 'demand']].to_csv("ans.csv", index=False)